In [ ]:
import os
import sys
sys.path.append(os.path.abspath('..'))

In [1]:
import os, utils
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from preprocessing.read_data import extract_metadata, extract_sequences, clean_sequences, clean_collection_dates, merge_dfs, remove_consensus_call_sequences

In [2]:
settings = utils.load_settings()
raw_data_dir = settings['raw_data_dir']
cleaned_data_dump_dir = settings['cleaned_data_dir']

In [3]:
# Read metadata
metadata_df = extract_metadata(raw_data_dir)
metadata_df.head()

,Accession ID,Lineage,Collection date,country,Submission date
0,EPI_ISL_1628772,B.1.1.353,2021-02-02,Reunion,2021-04-17
1,EPI_ISL_1166942,B.1.160.18,2021-01-18,Mayotte,2021-03-05
2,EPI_ISL_1018072,R.1,2021-01-10,Ghana,2021-02-12
3,EPI_ISL_729972,B.1.1,2020-07-07,Nigeria,2020-12-23
4,EPI_ISL_1700676,B.1.351,2021-01-12,Equatorial Guinea,2021-04-23


In [4]:
metadata_df = clean_collection_dates(metadata_df)

In [5]:
# Read sequences
sequences_df = extract_sequences(raw_data_dir)
sequences_df.head()

,Accession ID,sequence
0,EPI_ISL_16648364,CAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTT...
1,EPI_ISL_16698997,AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCT...
2,EPI_ISL_16698995,ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
3,EPI_ISL_16698994,ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...
4,EPI_ISL_16698989,ACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATC...


In [6]:
# Clean sequences
sequences_df = clean_sequences(sequences_df)

In [7]:
# Merge metadata and sequences
merged_df = merge_dfs(metadata_df, sequences_df)

# Clean lineages
merged_df = remove_consensus_call_sequences(merged_df)

In [9]:
merged_df

42037

In [12]:
# Save the merged dataframe
merged_df.to_parquet(os.path.join(cleaned_data_dump_dir, 'cleaned_dataset.parquet'), index=False)

In [13]:
# Save only the lineage column
table = pa.table(pd.DataFrame(merged_df['lineage']))
pq.write_table(table, os.path.join(cleaned_data_dump_dir, 'flat_labels.parquet'))